In [ ]:
import pandas as pd
from pymongo import MongoClient
import os

In [9]:
url = os.getenv("MONGODB_URL")
print(url)

In [10]:
# Step 1: MongoDB connection
client = MongoClient(url)
db = client["project"]


ConfigurationError: Empty host (or extra comma in host list)

In [5]:
# Step 2: Set the CSV folder path
csv_folder = "./dataset"

# Step 3: Loop through all CSV files in the folder
for filename in os.listdir(csv_folder):
    if filename.endswith(".csv"):
        file_path = os.path.join(csv_folder, filename)
        collection_name = os.path.splitext(filename)[0]  # Remove .csv extension

        print(f"📄 Loading '{filename}' → 🗂️ Collection: '{collection_name}'")

        # Load CSV and convert to dictionary
        df = pd.read_csv(file_path)
        records = df.to_dict("records")

        # Insert into MongoDB
        collection = db[collection_name]
        if records:
            collection.insert_many(records)
            print(f"✅ Inserted {len(records)} records into '{collection_name}'")
        else:
            print(f"⚠️ No data found in '{filename}'")

print("🎉 All CSVs have been imported successfully!")

📄 Loading 'pesticides.csv' → 🗂️ Collection: 'pesticides'
✅ Inserted 4349 records into 'pesticides'
📄 Loading 'rainfall.csv' → 🗂️ Collection: 'rainfall'
✅ Inserted 6727 records into 'rainfall'
📄 Loading 'temp.csv' → 🗂️ Collection: 'temp'
✅ Inserted 71311 records into 'temp'
📄 Loading 'yield.csv' → 🗂️ Collection: 'yield'


: 